In [1]:
import pandas as pd

import osmosys.osmosys
import osmosys.Backups
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(osmosys.Backups)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\proyectos\\OSMOSYS_2023\\IntegracionAI2023\\AI_OsmosysIntegrationScripts2023\\model\\modelAI.py'>

In [38]:
## parameters
indicatorCodeAI = 'AT1_01'
month = 'ENERO'
month_number = '2023-01'
year = 2023
test = False
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')


--------------------------------------AT1_01------------------------------------------------------


In [39]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf.info)
indicatorIdsOsmosys = subformDf.osmosys_indicator_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicador_ai_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: cfokrg2ldnik44oigs
parentFormIdAI: cgdo4q4ldnik44oig6
indicatorIdAI: c7ot7kflcza9pfw5x
indicatorIdsOsmosys: [159]


In [40]:
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))


indicatorIdsOsmosys: [159]
reported data: 6


In [41]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)

In [42]:
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    poblacion_meta = ["Refugiados/as y migrantes", "Comunidad de acogida"]
    AT1_01_RM_NA_PR = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    AT1_01_RM_NN_PR = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    AT1_01_RM_MM_PR = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    AT1_01_RM_HH_PR = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    AT1_01_RM_OTR_PR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)
    AT1_01_CA_NA_PR = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    AT1_01_CA_NN_PR = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    AT1_01_CA_MM_PR = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    AT1_01_CA_HH_PR = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    AT1_01_CA_OTR_PR = int(dfCa.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[
        0].value_a

    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                   cantonCode=cantonCode)
    AT1_01_RM_LGBT_PR = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)

    AT1_01_CA_LGBT_PR = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)

    subform = model.modelAI.SubFormAT1_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        poblacion_meta=poblacion_meta,
        poblacion_meta_freq='Primera vez',
        modalidad_impl='c3jf6gykykwwemuk7', ##'Provisión de servicio',
        AT1_01_RM_NA_PR=AT1_01_RM_NA_PR,
        AT1_01_RM_NN_PR=AT1_01_RM_NN_PR,
        AT1_01_RM_MM_PR=AT1_01_RM_MM_PR,
        AT1_01_RM_HH_PR=AT1_01_RM_HH_PR,
        AT1_01_RM_OTR_PR=AT1_01_RM_OTR_PR,
        AT1_01_RM_LGBT_PR=AT1_01_RM_LGBT_PR,
        AT1_01_CA_NA_PR=AT1_01_CA_NA_PR,
        AT1_01_CA_NN_PR=AT1_01_CA_NN_PR,
        AT1_01_CA_MM_PR=AT1_01_CA_MM_PR,
        AT1_01_CA_HH_PR=AT1_01_CA_HH_PR,
        AT1_01_CA_OTR_PR=AT1_01_CA_OTR_PR,
        AT1_01_CA_LGBT_PR=AT1_01_CA_LGBT_PR
    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 6


In [43]:
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)


In [44]:
## creo respaldo
osmosys.Backups.do_backup(indicatorCodeAI=indicatorCodeAI, indicatorIdsOsmosys= indicatorIdsOsmosys, month=month, year=year, changesList=changesList, finalJson=finalJson)

<class 'list'>
 creado respaldo: 2023-ENERO\data_AT1_01-[159].json


In [45]:
## envio a AI
if (test):
    print(
        '--------------------------------------' + indicatorCodeAI + '-' + str(
            indicatorIdsOsmosys) + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '-' + str(
                    indicatorIdsOsmosys) + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

se envia a AI
--------------------------------------AT1_01-[159]------------------------------------------------------
